In [2]:
#Biblioteques et fonctions

import re
import glob
import json
import nltk

def lire_fichier(chemin):
    f = open(chemin, encoding="utf-8")
    chaine = f.read()
    f.close()
    return chaine

def decouper_mots(chaine):
    #liste_mots = chaine.split()
    liste_mots = nltk.word_tokenize(chaine, language='french') # Exercice 4 : Du texte au token
    return liste_mots

def afficher_contextes(chaine, terme, taille_contexte = 10):
    match = re.search(terme, chaine)
    contexts = []
    while match is not None:
        #Les bornes gauche et droite autour du mot :
        gauche = max(match.start()-1, 0)-taille_contexte
        droite =  match.end()+1+taille_contexte
        contexts.append(chaine[gauche:droite])
        chaine = chaine[match.end():]
        match = re.search(terme, chaine)
    for c in contexts:
        print("...", c, "...")


def afficher_chemins(dic_chemins_trouves, N=10):

# afficher la liste des documents en question
# afficher si aucun mot de la requête n'est connu
# ne pas afficher les doublons
# mettre les documents dans l'ordre décroissant du nombre de mots de la requête qu'il contienne

    if len(chemins_trouves) == 0:
        print("J'ai pas trouvé des documents :(")
        return 0
    
    if N == 0:
        N = len(dic_chemins_trouves)
    liste_chemins_trouves = [[freq, chemin] for chemin, freq in dic_chemins_trouves.items()]
    liste_chemins_trouves_tri = sorted(liste_chemins_trouves, reverse=True)    
    
    print("- J'ai trouvé {} documents, voici les {} premiers -".format(len(liste_chemins_trouves_tri), N))
    for freq, chemin in liste_chemins_trouves_tri[:N]:
        print("-> {} - {}".format(freq, chemin))
        

def afficher_chemins_avec_contextes(dic_chemins_trouves, requete_mots, N=30):

    if len(chemins_trouves) == 0:
        print("J'ai pas trouvé des documents :(")
        return 0
    
    if N == 0:
        N = len(dic_chemins_trouves)
    liste_chemins_trouves = [[freq, chemin] for chemin, freq in dic_chemins_trouves.items()]
    liste_chemins_trouves_tri = sorted(liste_chemins_trouves, reverse=True)    
    
    print("- J'ai trouvé {} documents, voici les {} premiers -".format(len(liste_chemins_trouves_tri), N))
    for freq, chemin in liste_chemins_trouves_tri[:N]:
        chaine = lire_fichier(chemin)
        print("-> {} - {}".format(freq, chemin))
        
        for mot in requete_mots:
            print(">",mot,"<")
            afficher_contextes(chaine, mot)  

        
      

In [3]:
# Exercice 1 : Construire un index

index = {}

for chemin in glob.glob("../TD4/corpus_multi/fr/*/*"):
    chaine = lire_fichier(chemin)
    mots = decouper_mots(chaine)
    for mot in mots:
        if mot not in index:
            index[mot] = set()#pour ne pas avoir de doublons
        index[mot].add(chemin)

#print(list(index.keys())[:50]) # 50 mots dans l'index
#print(index["indique"]) # documents qui contient le mot "indique"
for mot in ["indique", "européenne", "toto"]:
    if mot in index:
        num_docs = len(index[mot])
    else:
        num_docs = 0
    print("{} : {} documents.".format(mot, num_docs))
    
    

indique : 26 documents.
européenne : 262 documents.
toto : 0 documents.


In [99]:
index = {mot:list(liste_fichiers) for mot, liste_fichiers in index.items()}

w = open("index.json", "w")
w.write(json.dumps(index))
w.close()

In [100]:
# Exercice 2 : Requêter le corpus

f = open("index.json", "r")
index = json.load(f)
f.close()

requete = "Commission Européenne" # créer une variable qui contient la requête, par exemple "Commission Européenne"
requete_mots = decouper_mots(requete) # découper la requête en mots

dic_chemins_trouves ={} 

# chercher pour chaque mot les documents où il apparaît
for mot in requete_mots: 
    if mot in index:
        for chemin in index[mot]:
            if chemin in dic_chemins_trouves:
                dic_chemins_trouves[chemin] += 1
            else:
                dic_chemins_trouves[chemin] = 1


afficher_chemins(dic_chemins_trouves, 10)



- J'ai trouvé 268 documents, voici les 10 premiers -
-> 2 - ../TD4/corpus_multi/fr/test/2009-11-20_celex_IP-09-1779.fr.html
-> 2 - ../TD4/corpus_multi/fr/appr/2009-09-18_celex_IP-09-1334.fr.html
-> 2 - ../TD4/corpus_multi/fr/appr/2009-06-26_celex_IP-09-1043.fr.html
-> 2 - ../TD4/corpus_multi/fr/appr/2009-06-25_celex_IP-09-1021.fr.html
-> 2 - ../TD4/corpus_multi/fr/appr/2009-04-27_celex_IP-09-640.fr.html
-> 2 - ../TD4/corpus_multi/fr/appr/2009-04-02_celex_IP-09-526.fr.html
-> 1 - ../TD4/corpus_multi/fr/test/2009-12-31_celex_IP-09-2002.fr.html
-> 1 - ../TD4/corpus_multi/fr/test/2009-12-21_celex_IP-09-1988.fr.html
-> 1 - ../TD4/corpus_multi/fr/test/2009-12-07_celex_IP-09-1884.fr.html
-> 1 - ../TD4/corpus_multi/fr/test/2009-12-07_celex_IP-09-1883.fr.html


In [101]:
# Exercice 3 : Présenter les contextes
print(requete_mots)
afficher_chemins_avec_contextes(dic_chemins_trouves, requete_mots, 10)

['Commission', 'Européenne']
- J'ai trouvé 268 documents, voici les 10 premiers -
-> 2 - ../TD4/corpus_multi/fr/test/2009-11-20_celex_IP-09-1779.fr.html
> Commission <
... e 2009  La Commission adopte le  ...
... e 2010  La Commission européenne ...
... mbre de la Commission chargé de  ...
... née par la Commission . L'aide e ...
... fient à la Commission en début d ...
> Européenne <
... de l'Union Européenne vivrait en ...
-> 2 - ../TD4/corpus_multi/fr/appr/2009-09-18_celex_IP-09-1334.fr.html
> Commission <
... rd'hui, la Commission européenne ...
... mbre de la Commission européenne ...
... ropéen, la Commission européenne ...
...   FVG Film Commission   IT   Kor ...
> Européenne <
... ssociation Européenne du Film d' ...
-> 2 - ../TD4/corpus_multi/fr/appr/2009-06-26_celex_IP-09-1043.fr.html
> Commission <
... oblème, la Commission européenne ...
... dent de la Commission, a officie ...
... mation. La Commission européenne ...
> Européenne <
... de l'Ecole Européenne de Bruxell ...
